In [ ]:
from cedne import utils

In [ ]:
w = utils.makeWorm()
nn = w.networks["Neutral"]
# utils.loadNeuropeptides(nn)
# utils.loadNeurotransmitters(nn)
# utils.loadGapJunctions(nn)

In [ ]:
nn.neurons['SAADR'].type

# Get all inputs to a neuron.

In [ ]:
## Shell graph here.
center = nn.neurons['RIR']
sub_edges = center.get_connections()
nn_sub = nn.subnetwork(connections=sub_edges)

asel = utils.plot_shell(nn_sub, center=center, figsize=(6,6), fontsize=10, save=True)

In [ ]:
## Shell graph here.
center = nn.neurons['URYVR']
sub_edges = center.get_connections()
nn_sub = nn.subnetwork(connections=sub_edges)

asel = utils.plot_shell(nn_sub, center=center, figsize=(6,6), fontsize=10, save=True)

## Inputs to two neurons.

In [ ]:
sub_edges

In [ ]:
import networkx as nx

## Shell graph here.
center = [nn.neurons['AWCL'], nn.neurons['AWCL'], nn.neurons['ASEL'], nn.neurons['ASER']]
sub_edges = []
layered_neurons = {0: center, 1:[], 2:[]}
for n in center:
    if len(n.get_connections())>2:
        neurs_interest = n.get_connections()
         
        neurs_int = [e for e in neurs_interest if e[1].type == 'interneuron' and nn.connections[e].weight>1]
        sub_edges+= neurs_int
        # Define the layers
        sens = [e[1] for e in neurs_interest if e[1].type == 'sensory' and nn.connections[e].weight>1]
        ints = [e[1] for e in neurs_interest if e[1].type == 'interneuron' and nn.connections[e].weight>1]
        motors = [e[1] for e in neurs_interest if e[1].type == 'motorneuron' and nn.connections[e].weight>1] 

        sens = [e[1] for e in neurs_interest if e[1].type == 'sensory' and nn.connections[e].weight>1]
        ints = [e[1] for e in neurs_interest if e[1].type == 'interneuron' and nn.connections[e].weight>1]
        motors = [e[1] for e in neurs_interest if e[1].type == 'motorneuron' and nn.connections[e].weight>1] 

        # Create a dictionary to store the neurons in each layer
        layered_neurons[1]+=sens
        layered_neurons[1]+=ints
        layered_neurons[2]+=motors
        #[nn.neurons[neuron] for neuron in neurons] for layer, neurons in layers.items()}

        # Create a NetworkX graph
G = nx.DiGraph()

# Add nodes with the layer attribute
for layer, neurons in layered_neurons.items():
    for neuron in neurons:
        print(neuron)
        G.add_node(neuron.name, layer=layer)

# Add edges
for pre, post, _ in sub_edges:
    G.add_edge(pre.name, post.name)

# Define the position of nodes for plotting
pos = {}
for layer, neurons in layered_neurons.items():
    for i, neuron in enumerate(neurons):
        pos[neuron.name] = (layer, i)

# Draw the network
plt.figure(figsize=(8, 8))
nx.draw(G, pos, with_labels=True, node_size=5000, node_color='lightblue', font_size=10, font_weight='bold')
# plt.savefig('ADA_layered.svg')
plt.show()
    # nn_sub = nn.subnetwork(connections=sub_edges)

# ada = utils.plot_shell(nn_sub, center=center, figsize=(8,8), fontsize=10, save='ADA.svg')
# ada = utils.plot_layered(sub_edges, nn_sub)#, figsize=(8,8), fontsize=10, save='ADA.svg')

# Get all paths between two neurons.

## Print all direct paths between two neurons. This could be useful for circuit analysis experiments or to generate molecular screens.

In [ ]:
w2 = utils.makeWorm(chem_only=True)
nn2 = w2.networks["Neutral"]

In [ ]:
neurons_of_interest = ['AWCL', 'RIAL']
n1 = nn2.neurons[neurons_of_interest[0]]
n2 = nn2.neurons[neurons_of_interest[1]]
print("||Neuron 1|| Neuron 2|| Connection type || Putative connection details||")
print("=================================")
for e,c in n1.get_connections(n2).items():
    if c.connection_type == 'chemical-synapse':
        print(f"||{c.pre.name} || {c.post.name} || {c.connection_type} || " + f"{c.putative_neurotrasmitter_receptors} ||")
    elif c.connection_type == 'gap-junction':
        print(f"||{c.pre.name} || {c.post.name} || {c.connection_type} || " + f"{c.putative_gapjn_subunits} ||")
    else:
        print(f"||{c.pre.name} || {c.post.name} || {c.connection_type} ||" )

In [ ]:
center = [n1, n2]
path_length = 2
conn_path = n1.paths_to(n2, path_length=path_length) + n2.paths_to(n1, path_length=path_length)
all_steps_on_path = [conn_ for conn in conn_path for conn_ in conn]

In [ ]:
connection_subnetwork = nn2.subnetwork(connections=all_steps_on_path)
edge_color={}
for e,c in connection_subnetwork.connections.items():
    if c.connection_type == 'chemical-synapse':
        if e[0].name == n1.name and e[1].name == n2.name:
            edge_color[e] = 'gray'
        else:
            edge_color[e] = 'lightgray'
        # if len(c.putative_neurotrasmitter_receptors):
        #     if c.putative_neurotrasmitter_receptors[0][0] == 'Glutamate':
        #         edge_color[e] = 'gray'
        #     else:
        #         edge_color[e] = 'gray'
        # else:
        #     edge_color[e] = 'gray' 
    else:
        edge_color[e] = 'gray' 
utils.plot_shell(connection_subnetwork, center=center, figsize=(4.5,5), fontsize=10, edge_color_dict=edge_color, save='AWCL_RIAL_indirect_paths.svg')

In [ ]:
n2.name

In [ ]:
all_n1_paths = n2.all_paths(direction='in', path_length =1)

In [ ]:
print(all_n1_paths)

## Print paths that are at most N steps removed.

In [ ]:
center = [n1, n2]
path_length = 3
conn_path = n1.paths_to(n2, path_length=path_length) + n2.paths_to(n1, path_length=path_length)
all_steps_on_path = [conn_ for conn in conn_path for conn_ in conn]
connection_subnetwork = nn.subnetwork(connections=all_steps_on_path)

In [ ]:
edge_color = {}
for e,c in connection_subnetwork.connections.items():
    if e[0].name in neurons_of_interest and e[1].name in neurons_of_interest:
        edge_color[e] = 'purple'
        continue
    if c.connection_type == 'chemical-synapse':
        edge_color[e] = 'gray' 
        #if len(c.putative_neurotrasmitter_receptors):
        #    if c.putative_neurotrasmitter_receptors[0][0] == 'Glutamate':
        #        edge_color[e] = 'gray'
        #    else:
        #        edge_color[e] = 'gray'
        #else:
        #    edge_color[e] = 'gray'
    else:
        edge_color[e] = 'gray' 

In [ ]:
awcl_rial = utils.plot_shell(connection_subnetwork, center=center, figsize=(6,6), fontsize=10, edge_color_dict=edge_color)#, save='AWCL_RIAL_indirect_paths.svg')

## Color by neurotransmitter type

In [ ]:
for e,c in connection_subnetwork.connections.items():
    if c.connection_type == 'chemical-synapse':
        if len(c.putative_neurotrasmitter_receptors):
            if c.putative_neurotrasmitter_receptors[0][0] == 'Glutamate':
                edge_color[e] = 'green'
            else:
                edge_color[e] = 'gray'
        else:
            edge_color[e] = 'gray'
    else:
        edge_color[e] = 'gray' 

utils.plot_shell(connection_subnetwork, center=center, figsize=(6,6), fontsize=10, edge_color_dict=edge_color)

In [ ]:
## Make a matrix of connections vs connection types and do a PCA analysis to see if there are groups of connections across
## the matrix

In [ ]:
len_out_dict = {}
for n in nn.neurons:
    len_out_dict[n] = len(nn.neurons[n].out_connections)

In [ ]:
nn.neurons['AVAL'].outgoing()

In [ ]:
for n in nn.neurons:
    if nn.neurons[n].category == 'SN3':
        print(n, [m[1].name for m, out_n in nn.neurons[n].out_connections.items() if m[1].category=='SN6'])

In [ ]:
all_NPs = {n:[] for n in nn.neurons.keys()}
for e,c in nn.connections.items():
    if c.connection_type not in ['chemical-synapse', 'gap-junction']:
        all_NPs[e[0].name].append(c.connection_type)

count_NPs = {n:[] for n in nn.neurons.keys()}
for n in all_NPs:
    all_NPs[n] = set(all_NPs[n])
    count_NPs[n] = len(all_NPs[n])

In [ ]:
print(dict(sorted(count_NPs.items(), key=lambda item: item[1], reverse=True)))